In [42]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.display import Image
sns.set()

In [43]:
# pip install matplotlib_venn --upgrade --user

https://www.glerl.noaa.gov/metdata/mil/archive/ 

In [44]:
# pip install pandas --upgrade --user

In [45]:
# pip install numexpr --upgrade --user

In [46]:
# pip install seaborn --upgrade --user

In [47]:
# pip install scipy --upgrade --user

The first data set that will be evaluated in this notebook is stored in `mkecallswheader.csv`. This dataset comes from requesting the bulk data option from a [website](https://mpd.digitalpublicworks.com/?start=2019-01-05T00:00:00-06:00&end=2019-01-05T23:59:59.999999-06:00) that scrapes the milwaukee police department call logs found [here](https://itmdapps.milwaukee.gov/MPDCallData/) and stores them. This data is stored in a postgres server. The official .gov site shows that the data should have headers of call number, date/time, location, police district, nature of call, and status. The bulk data stored in the .csv file has a couple extra headers of id, inserted_at, updated_at, and point. These features will need to be dropped later on since they do not pertain to the data itself and are an artifact of how the data was stored.

In [48]:
original_data = pd.read_csv("mkecallswheader.csv")

In [49]:
original_data.head(10)

,id,time,location,district,nature,status,inserted_at,updated_at,point,call_id
0,2093116,2019-05-21 15:19:03,"7420 W GOOD HOPE RD,MKE",4,ACC PI,Service in Progress,2019-05-21 20:51:09,2019-05-21 20:51:09,0101000020E6100000FC7C94111793454061D971683600...,191411633
1,2093127,2019-05-21 15:24:30,"1421 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,2019-05-21 20:57:11,2019-05-21 20:57:11,0101000020E6100000D2AB014A4386454067C416CCA9FC...,191411672
2,2093141,2019-05-21 15:25:46,"4054 N 71ST ST,MKE",7,SUBJ WANTED,Assignment Completed,2019-05-21 21:00:12,2019-05-21 21:00:12,0101000020E610000053FFC5D8AE8B45402CAE3B270700...,191411674
3,2093805,2019-05-21 20:46:28,"245 W LINCOLN AV,MKE",2,SPECIAL ASSIGN,Service in Progress,2019-05-22 02:22:32,2019-05-22 02:22:32,0101000020E610000078ABF8D04F804540633ABE0779FA...,191412545
4,2093816,2019-05-21 20:50:03,"1721 W CANAL ST,MKE",3,TRBL W/SUBJ,Unable to Locate Complainant,2019-05-22 02:25:33,2019-05-22 02:25:33,0101000020E6100000E8323509DE834540C3D7D7BAD4FB...,191412465
5,2093829,2019-05-21 21:02:37,"E WRIGHT ST / N WEIL ST,MKE",5,PARK AND WALK,Service in Progress,2019-05-22 02:37:36,2019-05-22 02:37:36,0101000020E6100000DEF1DC312B88454059D878558CF9...,191412584
6,2093872,2019-05-21 20:50:47,"9010 N 95TH ST,MKE",4,WELFARE CITIZEN,Advised,2019-05-22 02:52:43,2019-05-22 02:52:43,0101000020E6100000357D76C07597454080B4FF01D601...,191412544
7,2093887,2019-05-21 21:25:33,"983 W ARTHUR AV,MKE",2,BATTERY DV,Service in Progress,2019-05-22 03:01:48,2019-05-22 03:01:48,0101000020E6100000BC033C69E17F454041ABDDC02EFB...,191412632
8,2093918,2019-05-21 21:36:05,"4115 N 56TH ST,MKE",7,RETURN STATION,Assignment Completed,2019-05-22 03:16:51,2019-05-22 03:16:51,0101000020E610000014483FD0C08B45404EB747CAF1FE...,191412656
9,2093929,2019-05-21 21:45:53,"7806 W HAMPTON AV,MKE",7,TRAFFIC STOP,Advised,2019-05-22 03:22:52,2019-05-22 03:22:52,0101000020E6100000F6F0C05B7B8D45404F34B4A69E00...,191412676


In [50]:
original_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027695 entries, 0 to 4027694
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   id           4027695 non-null  int64 
 1   time         4027695 non-null  object
 2   location     4027695 non-null  object
 3   district     3937463 non-null  object
 4   nature       4027695 non-null  object
 5   status       4027695 non-null  object
 6   inserted_at  4027695 non-null  object
 7   updated_at   4027695 non-null  object
 8   point        3958766 non-null  object
 9   call_id      4027695 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 307.3+ MB


In [51]:
original_data.describe()

,id,call_id
count,4.027695e+06,4.027695e+06
mean,5.589211e+06,1.891808e+08
std,2.392488e+07,1.459309e+07
min,1.000000e+00,1.630815e+08
25%,1.006924e+06,1.732129e+08
50%,2.013848e+06,1.906707e+08
75%,3.020772e+06,2.016115e+08
max,1.635417e+08,2.201010e+08


From the above calls to the [`.head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html), [`.info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html), and [`.describe()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) methods it is possible to see that there are 10 total features. Of these features there are two features that are formatted as integers and eight features formatted as the default object that pandas imports non-numerical features as. From the object classified features there are a few that can be converted to new types. The time column should be formatted as a date time object. District should be converted to a numerical categorical value. Nature and status should be converted to categorical features. Location should be kept as an object feature. More features should be extracted from the location in order to draw further observations. Both street name and street suffix would be good features to extract.

The first step is to copy the data into a new dataframe so that the original data can be preserved

In [52]:
df = original_data.copy(deep=True)

All of the postgres features that are not part of the data can be dropped in the next step. These are the headers of id, inserted_at, updated_at, and point.

In [53]:
df = df.drop('id', axis=1)
df = df.drop('inserted_at', axis=1)
df = df.drop('updated_at', axis=1)
df = df.drop('point', axis=1)

Now the time feature can be converted into a date time object.

In [54]:
df['datetime'] = pd.to_datetime(df['time'], infer_datetime_format=True)
df = df.drop('time', axis=1)

In [55]:
df.head()

,location,district,nature,status,call_id,datetime
0,"7420 W GOOD HOPE RD,MKE",4,ACC PI,Service in Progress,191411633,2019-05-21 15:19:03
1,"1421 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,191411672,2019-05-21 15:24:30
2,"4054 N 71ST ST,MKE",7,SUBJ WANTED,Assignment Completed,191411674,2019-05-21 15:25:46
3,"245 W LINCOLN AV,MKE",2,SPECIAL ASSIGN,Service in Progress,191412545,2019-05-21 20:46:28
4,"1721 W CANAL ST,MKE",3,TRBL W/SUBJ,Unable to Locate Complainant,191412465,2019-05-21 20:50:03


In [56]:
# https://stackoverflow.com/a/43427677 for arguments to get null counts
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027695 entries, 0 to 4027694
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype         
---  ------    --------------    -----         
 0   location  4027695 non-null  object        
 1   district  3937463 non-null  object        
 2   nature    4027695 non-null  object        
 3   status    4027695 non-null  object        
 4   call_id   4027695 non-null  int64         
 5   datetime  4027695 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 184.4+ MB


In [57]:
df.describe()

,call_id
count,4.027695e+06
mean,1.891808e+08
std,1.459309e+07
min,1.630815e+08
25%,1.732129e+08
50%,1.906707e+08
75%,2.016115e+08
max,2.201010e+08


The nature, status, and district features will now be converted into categorical features

In [58]:
df['district'].unique()

array(['4', '3', '7', '2', '5', '6', '1', nan, 'OCOE', 'OUT', 'NTF',
       'SPD', 'CITY', 'ICS3', 'CIB', 'DPR', 'TRU', 'SCD', 'ICS', 'SF',
       'NLA', 'ICS1', 'IFC', 'FI', 'SID', 'D0', 'ICS7', 'ICS5', 'JUNE',
       'DDAC', 'MIRT', 'ICS6', 'ID', 'ADMN', 'TEU', 'MID', 'NID'],
      dtype=object)

In [59]:
df['district'].value_counts()

3       673002
7       650109
5       567958
4       552701
2       545253
6       473525
1       448786
CITY      7080
NTF       6274
SPD       5477
OCOE      3316
OUT       2249
DDAC       399
SCD        300
DPR        277
ICS1       229
CIB        218
FI          83
SF          55
NLA         34
ICS3        32
SID         31
ICS         23
TEU         16
ICS5         9
JUNE         5
ICS6         4
IFC          4
TRU          3
D0           2
MIRT         2
ADMN         2
NID          2
ICS7         1
ID           1
MID          1
Name: district, dtype: int64

Running a value counts on the different districts shows that there are more than the expected number of police districts in the city of Milwaukee present in the data. There should only be districts one through seven. Instead the data contains more districts than expected. These districts will be converted into categorical anyways as their entries will be useful for some observations. The erraneous districts will likely be ignored when drawing district based conclusions as it is not known what the other districts mean. District will be converted to a categorical with 36 different categories.

In [60]:
df['nature'].unique()

array(['ACC PI', 'TRAFFIC STOP', 'SUBJ WANTED', 'SPECIAL ASSIGN',
       'TRBL W/SUBJ', 'PARK AND WALK', 'WELFARE CITIZEN', 'BATTERY DV',
       'RETURN STATION', 'BUSINESS CHECK', 'SUSPICIOUS-OTH', 'REPORTS',
       'THEFT', 'VIOL REST ORDER', 'SUBJ WITH GUN', 'TAVERN CHECK',
       'PATROL', 'FOLLOW UP', 'ACC PDO', 'TRAFFIC HAZARD',
       'FAMILY TROUBLE', 'NON PURSUIT', 'BATTERY', 'INVESTIGATION',
       'PRISONER TRANS', 'BUS INV', 'CALL FOR POLICE', 'THEFT VEHICLE',
       'SUSP PERS/AUTO', 'ENTRY TO AUTO', 'ASSIGNMENT', 'PROPERTY DAMAGE',
       'IND EXPOSURE', 'NOISE NUISANCE', 'PROPERTY PICKUP', 'SOLICITING',
       'STOLEN VEHICLE', 'OUT OF SERVICE', 'COURT DUTY', 'ENTRY',
       'PARKING TROUBLE', 'GRAFFITI', 'TRBL W/JUV', 'SHOTS FIRED',
       'SHOTSPOTTER', 'BUS INVESTIGATIO', 'THREAT', 'SUBJ WITH WEAPON',
       'TRAFFIC LASER', 'HOME VISIT DV', 'COMMUNITY MTNG',
       'CITIZEN CONTACT', 'RECK USE OF WEAP', 'ABAND/STOLEN PRO',
       'ASSIGN-ADMN MPD', 'CONVEY PROPERTY',

In [61]:
df['nature'].value_counts()

TRAFFIC STOP        401644
BUSINESS CHECK      325863
TRBL W/SUBJ         237324
RETURN STATION      209934
FOLLOW UP           150661
                     ...  
ACC PD1                  1
ABAND/LOST               1
2831 N 21ST              1
THREAT TO SCHOOL         1
STOLEN PROP              1
Name: nature, Length: 317, dtype: int64

The natures all look normal. They will be converted into a categorical value which will have 317 categories.

In [62]:
df['status'].unique()

array(['Service in Progress', 'City Citation(s) Issued',
       'Assignment Completed', 'Unable to Locate Complainant', 'Advised',
       'To be Filed', 'Advised/Referral', 'No Prosecution',
       'Open Investigation', 'Cleared by Arrest', 'False Alarm',
       'Filed Driver Exchange Report', 'Patrol Request', 'Referral',
       'Ordered to Appear', 'State Citation(s) Issued',
       'False Alarm (Weather Related)'], dtype=object)

In [63]:
df['status'].value_counts()

Service in Progress              1424568
Assignment Completed             1186663
Advised                           615111
Unable to Locate Complainant      336406
To be Filed                       164199
City Citation(s) Issued           153937
Advised/Referral                   87051
Open Investigation                 40803
No Prosecution                      6475
Cleared by Arrest                   4696
False Alarm                         2889
Filed Driver Exchange Report        2631
Referral                            1291
Patrol Request                       697
State Citation(s) Issued             160
False Alarm (Weather Related)         93
Ordered to Appear                     25
Name: status, dtype: int64

Status looks normal as well and will be converted to a categorical value. It will contain 17 different categories.

In [64]:
df['district'] = df['district'].astype("category")
df['nature'] = df['nature'].astype("category")
df['status'] = df['status'].astype('category')

In [65]:
df['district'].dtype

CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7', 'ADMN', 'CIB', 'CITY',
                  'D0', 'DDAC', 'DPR', 'FI', 'ICS', 'ICS1', 'ICS3', 'ICS5',
                  'ICS6', 'ICS7', 'ID', 'IFC', 'JUNE', 'MID', 'MIRT', 'NID',
                  'NLA', 'NTF', 'OCOE', 'OUT', 'SCD', 'SF', 'SID', 'SPD',
                  'TEU', 'TRU'],
, ordered=False)

In [66]:
df['nature'].dtype

CategoricalDtype(categories=['.', '0', '1 BLOCK NORTH OF', '1301', '1359', '1603', '1733',
                  '230 N 37TH ST', '2532', '2831 N 21ST',
                  ...
                  'VIOL REST ORD', 'VIOL REST ORD-DV', 'VIOL REST ORDER',
                  'WATER MAIN BREAK', 'WATER MAIN BRK', 'WEAPON',
                  'WELFARE CHK', 'WELFARE CITIZEN', 'WIRES DOWN', '`'],
, ordered=False)

In [67]:
df['status'].dtype

CategoricalDtype(categories=['Advised', 'Advised/Referral', 'Assignment Completed',
                  'City Citation(s) Issued', 'Cleared by Arrest',
                  'False Alarm', 'False Alarm (Weather Related)',
                  'Filed Driver Exchange Report', 'No Prosecution',
                  'Open Investigation', 'Ordered to Appear', 'Patrol Request',
                  'Referral', 'Service in Progress',
                  'State Citation(s) Issued', 'To be Filed',
                  'Unable to Locate Complainant'],
, ordered=False)

The nature, status, and district features are now formatted as categories.

The last step is to extract features from the location category

In [68]:
def get_street_info(address: str) -> (int, str, str, int):
    """
    This method will take in a string representing an address and will return the information present in that address.
    Some example addresses are as follows:
        0             7420 W GOOD HOPE RD,MKE
        1                  1421 N 27TH ST,MKE
        2                  4054 N 71ST ST,MKE
        3                245 W LINCOLN AV,MKE
        4                 1721 W CANAL ST,MKE
        5         E WRIGHT ST / N WEIL ST,MKE
        6                  9010 N 95TH ST,MKE
    :param address: the string passed in representing the address
    :return: a tuple containing (home number, street name, street type)
    :auth: Grant Fass
    :since: 8 February 2022
    """
    street_type_lookup = ["ALY", "ANX", "ARC", "AVE", "BYU", "BCH", "BND", "BLF", "BLFS", "BTM", "BLVD", "BR", "BRG", "BRK", "BRKS", "BG", "BGS", "BYP", "CP", "CYN", "CPE",
                          "CSWY", "CTR", "CTRS", "CIR", "CIRS", "CLF", "CLFS", "CLB", "CMN", "CMNS", "COR", "CORS", "CRSE", "CT", "CTS", "CV", "CVS", "CRK", "CRES", "CRST", 
                          "XING", "XRD", "XRDS", "CURV", "DL", "DM", "DV", "DR", "EST", "ESTS", "EXPY", "EXT", "EXTS", "FALL", "FLS", "FRY", "FLD", "FLDS", "FLT", "FLTS", 
                          "FRD", "FRDS", "FRST", "FRG", "FRGS", "FRK", "FRKS", "FT", "FWY", "GDN", "GDNS", "GTWY", "GLN", "GLNS", "GRN", "GRNS", "GRV", "GRVS", "HBR", "HBRS", 
                          "HVN", "HTS", "HWY", "HL", "HLS", "HOLW", "INLT", "IS", "ISS", "ISLE", "JCT", "JCTS", "KY", "KYS", "KNL", "KNLS", "LK", "LKS", "LAND", "LNDG", "LN",
                          "LGT", "LGTS", "LF", "LCK", "LCKS", "LDG", "LOOP", "MALL", "MNR", "MNRS", "MDW", "MDWS", "MEWS", "ML", "MLS", "MSN", "MTWY", "MT", "MTN", "MTNS", 
                          "NCK", "ORCH", "OVAL", "OPAS", "PARK", "PKWY", "PASS", "PSGE", "PATH", "PIKE", "PNE", "PNES", "PL", "PLN", "PLNS", "PLZ", "PT", "PTS", "PRT", "PRTS", 
                          "PR", "RADL", "RAMP", "RNCH", "RPD", "RPDS", "RST", "RDG", "RDGS", "RIV", "RD", "RDS", "RTE", "ROW", "RUE", "RUN", "SHL", "SHLS", "SHR", "SHRS", 
                          "SKWY", "SPG", "SPGS", "SPUR", "SQ", "SQS", "STA", "STRA", "STRM", "ST", "STS", "SMT", "TER", "TRWY", "TRCE", "TRAK", "TRFY", "TRL", "TRLR", "TUNL",
                          "TPKE", "UPAS", "UN", "UNS", "VLY", "VLYS", "VIA", "VW", "VWS", "VLG", "VLGS", "VL", "VIS", "WALK", "WALL", "WAY", "WAYS", "WL", "WLS", "AV"]
    # this is used primarily for error checking
    unmatched_suffix = ""
    # remove the ,MKE suffix from the location if present
    address = address.removesuffix(",MKE")
    # Array containing the seperate addresses in the passed entry
    addresses = []
    # Check if the entry contains a / or not.
    # The presence of a / denotes the entry as a corner with two streets present
    # For example: N HUMBOLDT AV / E NORTH AV
    if ('/' in address):
        addresses = address.split(' / ')
    else:
        addresses = [address]
    
    # now perform opperations for each address
    # print(addresses)
    out = []
    for a in addresses:
        # Set up the values to be returned
        house_number = None
        street = None
        street_suffix = None
        # split appart the address on spaces
        s = a.split(' ')
        # Check if the first cell is a nueric. This would be the house number if it is a numeric
        if s[0].isnumeric():
            # house number present
            house_number = int(s[0])
            street = ' '.join(s[1:-1]) # use -1 since last index is exclusive
        else:
            street = ' '.join(s[0:-1])
        # update the street suffix based on the last entry in the array
        if s[-1] in street_type_lookup:
            street_suffix = s[-1]
        # add the entries into the return field
        out.append((house_number, street, street_suffix))
    return out

In [69]:
#  E WRIGHT ST / N WEIL ST,MKE
get_street_info(df['location'][5])

[(None, 'E WRIGHT', 'ST'), (None, 'N WEIL', 'ST')]

In [70]:
#  1421 N 27TH ST,MKE
get_street_info(df['location'][1])

[(1421, 'N 27TH', 'ST')]

In [71]:
df['location'].head(6)

0        7420 W GOOD HOPE RD,MKE
1             1421 N 27TH ST,MKE
2             4054 N 71ST ST,MKE
3           245 W LINCOLN AV,MKE
4            1721 W CANAL ST,MKE
5    E WRIGHT ST / N WEIL ST,MKE
Name: location, dtype: object

Some of the locations are corners of two streets and the others are single house or location addresses. Due to this it will be necessary to create a categorical feature denoting if the entry is a corner or not. The attributes of both locations will then be recorded as well. Because corner locations do not have house numbers attached these will not be included for the second location value. Overall the following features will be added:
- isCorner
- houseNumber
- primaryStreetName
- primaryStreetSuffix
- secondaryStreetName
- secondaryStreetSuffix

In [72]:
def get_street_data_as_array(location: []) -> [int, int, str, str, str, str]:
    """
    Method to take in a location that contains up to two streets and combine it into one list for output.
    The location is an array of tuples up to two in length.
    Each tuple will have 3 entries of the form (houseNumber, streetName, streetSuffix).
    If two entries are present then the location is a corner.
    The output list will be of the form [isCorner, houseNumber, primaryStreetName, primaryStreetSuffix, secondaryStreetName, secondaryStreetSuffix]
    :param location: an array of tuples up to two in length.
    :return: list will be of the form [isCorner, houseNumber, primaryStreetName, primaryStreetSuffix, secondaryStreetName, secondaryStreetSuffix]
    :auth: Grant Fass
    :since: 8 February 2022
    """
    
    if len(location) == 1:
        # is not corner
        location_vals = list(location[0])
        return [False, location_vals[0], location_vals[1], location_vals[2], None, None]
    else:
        # is corner
        primary_location_vals = list(location[0])
        secondary_location_vals = list(location[1])
        return [True, None, primary_location_vals[1], primary_location_vals[2], 
        secondary_location_vals[1], secondary_location_vals[2]]

In [73]:
get_street_data_as_array(get_street_info(df['location'][1]))

[False, 1421, 'N 27TH', 'ST', None, None]

In [74]:
get_street_data_as_array(get_street_info(df['location'][5]))

[True, None, 'E WRIGHT', 'ST', 'N WEIL', 'ST']

In [76]:
# tolist() is needed since the output is a ndarray of lists.
street_data = df["location"].map(get_street_info).map(get_street_data_as_array).tolist()

In [77]:
header = ["isCorner", "houseNumber", "primaryStreetName", "primaryStreetSuffix", "secondaryStreetName", "secondaryStreetSuffix"]
data = pd.DataFrame(street_data, columns=header)
data.head(20)

,isCorner,houseNumber,primaryStreetName,primaryStreetSuffix,secondaryStreetName,secondaryStreetSuffix
0,False,7420.0,W GOOD HOPE,RD,None,None
1,False,1421.0,N 27TH,ST,None,None
2,False,4054.0,N 71ST,ST,None,None
3,False,245.0,W LINCOLN,AV,None,None
4,False,1721.0,W CANAL,ST,None,None
5,True,NaN,E WRIGHT,ST,N WEIL,ST
6,False,9010.0,N 95TH,ST,None,None
7,False,983.0,W ARTHUR,AV,None,None
8,False,4115.0,N 56TH,ST,None,None
9,False,7806.0,W HAMPTON,AV,None,None


In [78]:
# data['houseNumber'] = data["houseNumber"].astype('int64')
data['primaryStreetSuffix'] = data['primaryStreetSuffix'].astype('category')
data['primaryStreetName'] = data['primaryStreetName'].astype('category')
data['secondaryStreetSuffix'] = data['secondaryStreetSuffix'].astype('category')
data['secondaryStreetName'] = data["secondaryStreetName"].astype('category')

In [79]:
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027695 entries, 0 to 4027694
Data columns (total 6 columns):
 #   Column                 Non-Null Count    Dtype   
---  ------                 --------------    -----   
 0   isCorner               4027695 non-null  bool    
 1   houseNumber            3426083 non-null  float64 
 2   primaryStreetName      4027695 non-null  category
 3   primaryStreetSuffix    3844881 non-null  category
 4   secondaryStreetName    549725 non-null   category
 5   secondaryStreetSuffix  531853 non-null   category
dtypes: bool(1), category(4), float64(1)
memory usage: 58.3 MB


In [80]:
data['isCorner'].value_counts()

False    3477970
True      549725
Name: isCorner, dtype: int64

The locations have now been properly extracted into seperate features and stored in their own array with proper data types. They must now be added back in to the overall data array. It is also possible to see that there are 549,725 corner locations and 3,477,970 locations that are not corners.

In [81]:
df.shape

(4027695, 6)

In [82]:
data.shape

(4027695, 6)

In [83]:
df = pd.concat([df, data], join='outer', axis=1)

In [84]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027695 entries, 0 to 4027694
Data columns (total 12 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   location               4027695 non-null  object        
 1   district               3937463 non-null  category      
 2   nature                 4027695 non-null  category      
 3   status                 4027695 non-null  category      
 4   call_id                4027695 non-null  int64         
 5   datetime               4027695 non-null  datetime64[ns]
 6   isCorner               4027695 non-null  bool          
 7   houseNumber            3426083 non-null  float64       
 8   primaryStreetName      4027695 non-null  category      
 9   primaryStreetSuffix    3844881 non-null  category      
 10  secondaryStreetName    549725 non-null   category      
 11  secondaryStreetSuffix  531853 non-null   category      
dtypes: bool(1), category(7), dat

In [85]:
df.head()

,location,district,nature,status,call_id,datetime,isCorner,houseNumber,primaryStreetName,primaryStreetSuffix,secondaryStreetName,secondaryStreetSuffix
0,"7420 W GOOD HOPE RD,MKE",4,ACC PI,Service in Progress,191411633,2019-05-21 15:19:03,False,7420.0,W GOOD HOPE,RD,NaN,NaN
1,"1421 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,191411672,2019-05-21 15:24:30,False,1421.0,N 27TH,ST,NaN,NaN
2,"4054 N 71ST ST,MKE",7,SUBJ WANTED,Assignment Completed,191411674,2019-05-21 15:25:46,False,4054.0,N 71ST,ST,NaN,NaN
3,"245 W LINCOLN AV,MKE",2,SPECIAL ASSIGN,Service in Progress,191412545,2019-05-21 20:46:28,False,245.0,W LINCOLN,AV,NaN,NaN
4,"1721 W CANAL ST,MKE",3,TRBL W/SUBJ,Unable to Locate Complainant,191412465,2019-05-21 20:50:03,False,1721.0,W CANAL,ST,NaN,NaN


The df and data DataFrame objects were combined on their columns with an inner join using [`pd.concat`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html). This took df of size (4027695, 6), and data of size (4027695, 6) and updated df to be size (4027695, 12). After extracting the location data and appending it, the location feature is now irrelevant as it is harder to read and is correlated with the new features. As such it will be dropped.

In [86]:
df = df.drop('location', axis=1)

In [87]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027695 entries, 0 to 4027694
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   district               3937463 non-null  category      
 1   nature                 4027695 non-null  category      
 2   status                 4027695 non-null  category      
 3   call_id                4027695 non-null  int64         
 4   datetime               4027695 non-null  datetime64[ns]
 5   isCorner               4027695 non-null  bool          
 6   houseNumber            3426083 non-null  float64       
 7   primaryStreetName      4027695 non-null  category      
 8   primaryStreetSuffix    3844881 non-null  category      
 9   secondaryStreetName    549725 non-null   category      
 10  secondaryStreetSuffix  531853 non-null   category      
dtypes: bool(1), category(7), datetime64[ns](1), float64(1), int64(1)
memory usage: 135.2 MB


At this point the MPD (Milwaukee Police Department) dataset is mostly done being cleaned. The next steps are to load the weather dataset, clean it, then combine it with the MPD dataset.

In [88]:
# TODO: determine where the weather dataset comes from and write up a markdown cell for it

In [89]:
weather_data_original = pd.read_csv("weather_data_1hr.csv")

In [90]:
weather_data_original.shape

(119112, 32)

The weather dataset has 119112 entries. At first this number seems small, but some quick math (divide by 24 hours per day, divide by 365 days per year) shows that the dataset covers approximately 14 years beginning at 2008.

In [91]:
weather_data_original.head(10)

,loc_id,date,time,isdaytime,tempC,tempF,windspeedMiles,windspeedKmph,winddirdegree,winddir16point,...,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex
0,1,2008-07-01,0,no,14,58,7,11,241,WSW,...,58,11,51,14,57,15,23,14,57,1
1,1,2008-07-01,100,no,14,58,7,12,246,WSW,...,58,10,51,14,57,15,25,14,57,1
2,1,2008-07-01,200,no,14,58,8,12,251,WSW,...,58,10,51,14,57,16,26,14,57,1
3,1,2008-07-01,300,no,15,58,8,13,256,WSW,...,58,10,50,14,57,17,27,14,57,1
4,1,2008-07-01,400,no,15,59,8,12,255,WSW,...,59,10,50,14,58,16,26,14,58,1
5,1,2008-07-01,500,no,15,59,7,12,253,WSW,...,59,10,50,15,58,15,24,15,58,1
6,1,2008-07-01,600,no,15,59,7,11,252,WSW,...,59,10,50,15,59,14,23,15,59,1
7,1,2008-07-01,700,no,17,63,7,12,253,WSW,...,63,11,51,17,63,14,23,17,63,1
8,1,2008-07-01,800,yes,19,66,8,13,254,WSW,...,66,11,53,19,66,15,23,19,66,5
9,1,2008-07-01,900,yes,21,69,9,14,254,WSW,...,69,12,54,21,69,15,24,21,69,6


In [92]:
weather_data_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119112 entries, 0 to 119111
Data columns (total 32 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   loc_id           119112 non-null  int64  
 1   date             119112 non-null  object 
 2   time             119112 non-null  int64  
 3   isdaytime        119112 non-null  object 
 4   tempC            119112 non-null  int64  
 5   tempF            119112 non-null  int64  
 6   windspeedMiles   119112 non-null  int64  
 7   windspeedKmph    119112 non-null  int64  
 8   winddirdegree    119112 non-null  int64  
 9   winddir16point   119112 non-null  object 
 10  weatherCode      119112 non-null  int64  
 11  weatherIconUrl   119112 non-null  object 
 12  weatherDesc      119112 non-null  object 
 13  precipMM         119112 non-null  float64
 14  precipInches     119112 non-null  float64
 15  humidity         119112 non-null  int64  
 16  visibilityKm     119112 non-null  int6

Immediatly recognizable from the dataset is that it contains many correlated features.
- tempC and tempF
- windspeedMiles and windspeedKmph
- winddirdegree and winddir16point
- precipMM and precipInches
- visibilityKm and visibilityMiles
- pressureMB and pressureInches
- HeatIndexC and HeatIndexF
- DewPointC and DewPointF
- WindChillC and WindChillF
- WindGustMiles and WindGustKmph
- FeelsLikeC and FeelsLikeF


Any of the features using imperial units will be dropped in favor of their metric counterparts. This is done in order to preserve as much information as possible since the imperial features are correlated with the metric features while also being less precise. The winddirdegree feature will be retained instead of the winddir16point for this same reason. The weatherIconUrl feature will also be dropped as it is not very descriptive of our overall data.

In [93]:
weather_data = weather_data_original.copy(deep=True)
weather_data = weather_data.drop('tempF', axis=1)
weather_data = weather_data.drop('windspeedMiles', axis=1)
weather_data = weather_data.drop('winddir16point', axis=1)
weather_data = weather_data.drop('precipInches', axis=1)
weather_data = weather_data.drop('visibilityMiles', axis=1)
weather_data = weather_data.drop('pressureInches', axis=1)
weather_data = weather_data.drop('HeatIndexF', axis=1)
weather_data = weather_data.drop('DewPointF', axis=1)
weather_data = weather_data.drop('WindChillF', axis=1)
weather_data = weather_data.drop('WindGustMiles', axis=1)
weather_data = weather_data.drop('FeelsLikeF', axis=1)
weather_data = weather_data.drop('weatherIconUrl', axis=1)

In [94]:
weather_data.shape

(119112, 20)

In [95]:
weather_data.head(25)

,loc_id,date,time,isdaytime,tempC,windspeedKmph,winddirdegree,weatherCode,weatherDesc,precipMM,humidity,visibilityKm,pressureMB,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex
0,1,2008-07-01,0,no,14,11,241,113,Clear,0.0,77,10,1016,3,14,11,14,23,14,1
1,1,2008-07-01,100,no,14,12,246,113,Clear,0.0,77,10,1016,10,14,10,14,25,14,1
2,1,2008-07-01,200,no,14,12,251,113,Clear,0.0,76,10,1015,17,14,10,14,26,14,1
3,1,2008-07-01,300,no,15,13,256,113,Clear,0.0,76,10,1015,24,15,10,14,27,14,1
4,1,2008-07-01,400,no,15,12,255,113,Clear,0.0,74,10,1016,22,15,10,14,26,14,1
5,1,2008-07-01,500,no,15,12,253,113,Clear,0.0,73,10,1016,19,15,10,15,24,15,1
6,1,2008-07-01,600,no,15,11,252,113,Clear,0.0,71,10,1016,17,15,10,15,23,15,1
7,1,2008-07-01,700,no,17,12,253,113,Clear,0.0,67,10,1016,27,17,11,17,23,17,1
8,1,2008-07-01,800,yes,19,13,254,116,Partly cloudy,0.0,62,10,1016,37,19,11,19,23,19,5
9,1,2008-07-01,900,yes,21,14,254,116,Partly cloudy,0.0,58,10,1015,48,21,12,21,24,21,6


In [96]:
weather_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119112 entries, 0 to 119111
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   loc_id         119112 non-null  int64  
 1   date           119112 non-null  object 
 2   time           119112 non-null  int64  
 3   isdaytime      119112 non-null  object 
 4   tempC          119112 non-null  int64  
 5   windspeedKmph  119112 non-null  int64  
 6   winddirdegree  119112 non-null  int64  
 7   weatherCode    119112 non-null  int64  
 8   weatherDesc    119112 non-null  object 
 9   precipMM       119112 non-null  float64
 10  humidity       119112 non-null  int64  
 11  visibilityKm   119112 non-null  int64  
 12  pressureMB     119112 non-null  int64  
 13  cloudcover     119112 non-null  int64  
 14  HeatIndexC     119112 non-null  int64  
 15  DewPointC      119112 non-null  int64  
 16  WindChillC     119112 non-null  int64  
 17  WindGustKmph   119112 non-nul

The features in the revised data are all the correct types except for date, isdaytime, and weatherDesc. Date needs to be converted into a datetime, isdaytime needs to be a boolean, and weatherDesc needs to be categorical

In [97]:
def isdaytime_to_boolean(val: str) -> bool:
    """
    method to convert a string into a boolean
    :param val: (str) no or yes
    :return: (bool) False if no, True if yes
    :auth: Grant Fass
    :since: 8 February 2022
    """
    if val == 'no':
        return False
    return True

In [98]:
weather_data['date'] = pd.to_datetime(weather_data['date'], infer_datetime_format=True)
weather_data['isdaytime'] = weather_data['isdaytime'].map(isdaytime_to_boolean).astype('bool')
weather_data['weatherDesc'] = weather_data['weatherDesc'].astype('category')

In [99]:
weather_data[['date', 'isdaytime', 'weatherDesc']].head(15)

,date,isdaytime,weatherDesc
0,2008-07-01,False,Clear
1,2008-07-01,False,Clear
2,2008-07-01,False,Clear
3,2008-07-01,False,Clear
4,2008-07-01,False,Clear
5,2008-07-01,False,Clear
6,2008-07-01,False,Clear
7,2008-07-01,False,Clear
8,2008-07-01,True,Partly cloudy
9,2008-07-01,True,Partly cloudy


lastly the time feature needs to be modified. Currently this feature begins with 0 representing midnight and increments by 100 per hour. The first entry at midnight each day resets to 0. Thus it will make more sense to convert this feature to represent hours more directly by dividing it by 100.

In [100]:
weather_data['time'] = weather_data['time'] / 100

In [101]:
weather_data['time'].head(5)

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
Name: time, dtype: float64

The next cell uses the date and time features to make one merged datetime feature. [This](https://stackoverflow.com/a/44648068) stackoverflow post helped with using apply to map across multiple features. [This](https://stackoverflow.com/a/17152848) stackoverflow post helped with replacing hours in a [timestamp](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.replace.html). [This](https://stackoverflow.com/a/25129655) stackoverflow post also provided some assistance. The original date and time features will be dropped after the features are merged.

In [102]:
weather_data['datetime'] = weather_data.apply(lambda t: t['date'].replace(hour=int(t['time'])), axis=1)
weather_data = weather_data.drop('date', axis=1)
weather_data = weather_data.drop('time', axis=1)
# weather_data = weather_data.set_index('datetime')
weather_data.head()

,loc_id,isdaytime,tempC,windspeedKmph,winddirdegree,weatherCode,weatherDesc,precipMM,humidity,visibilityKm,pressureMB,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex,datetime
0,1,False,14,11,241,113,Clear,0.0,77,10,1016,3,14,11,14,23,14,1,2008-07-01 00:00:00
1,1,False,14,12,246,113,Clear,0.0,77,10,1016,10,14,10,14,25,14,1,2008-07-01 01:00:00
2,1,False,14,12,251,113,Clear,0.0,76,10,1015,17,14,10,14,26,14,1,2008-07-01 02:00:00
3,1,False,15,13,256,113,Clear,0.0,76,10,1015,24,15,10,14,27,14,1,2008-07-01 03:00:00
4,1,False,15,12,255,113,Clear,0.0,74,10,1016,22,15,10,14,26,14,1,2008-07-01 04:00:00


In [ ]:
# TODO: describe what the weather data looks like here


The next step is to merge the weather data and MPD datasets. This merge will be done by concatinating whatever day and hour match between the two datasets. This should lead to df maintaining the same number of rows but adding on the number of columns in weather_data

Procedure psuedocode

for each entry in MPD data:
    target_year = df['time']

https://www.google.com/search?q=pandas+merge+dataframes+by+closest+time&rlz=1C1GCEA_enUS988US988&oq=pandas+merge+dataframes+on+closest&aqs=chrome.1.69i57j0i22i30.8168j0j7&sourceid=chrome&ie=UTF-8


https://www.geeksforgeeks.org/how-to-sort-a-pandas-dataframe-by-date/


https://www.geeksforgeeks.org/merge-two-pandas-dataframes-based-on-closest-datetime/

In [110]:
df = df.sort_values(by='datetime')
df.head()

,district,nature,status,call_id,datetime,isCorner,houseNumber,primaryStreetName,primaryStreetSuffix,secondaryStreetName,secondaryStreetSuffix
3159744,NaN,FOLLOW UP,Assignment Completed,163120850,2016-11-07 09:33:52,False,2920.0,N 4TH,ST,NaN,NaN
3159797,NaN,PROPERTY DAMAGE,Advised/Referral,163120774,2016-11-07 09:33:55,False,5929.0,W BLUE MOUND,RD,NaN,NaN
3159848,NaN,VACANT HOUSE CHK,Assignment Completed,163120851,2016-11-07 09:34:58,False,3425.0,N 93RD,ST,NaN,NaN
3159640,NaN,FOLLOW UP,Assignment Completed,163120853,2016-11-07 09:36:20,False,245.0,W LINCOLN,AV,NaN,NaN
3159691,NaN,TRAFFIC STOP,Advised,163120857,2016-11-07 09:38:10,False,765.0,N 25TH,ST,NaN,NaN


In [111]:
weather_data = weather_data.sort_values(by='datetime')
weather_data.head()

,loc_id,isdaytime,tempC,windspeedKmph,winddirdegree,weatherCode,weatherDesc,precipMM,humidity,visibilityKm,pressureMB,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex,datetime
0,1,False,14,11,241,113,Clear,0.0,77,10,1016,3,14,11,14,23,14,1,2008-07-01 00:00:00
1,1,False,14,12,246,113,Clear,0.0,77,10,1016,10,14,10,14,25,14,1,2008-07-01 01:00:00
2,1,False,14,12,251,113,Clear,0.0,76,10,1015,17,14,10,14,26,14,1,2008-07-01 02:00:00
3,1,False,15,13,256,113,Clear,0.0,76,10,1015,24,15,10,14,27,14,1,2008-07-01 03:00:00
4,1,False,15,12,255,113,Clear,0.0,74,10,1016,22,15,10,14,26,14,1,2008-07-01 04:00:00


In [114]:
print(df.shape)
print(weather_data.shape)

(4027695, 11)
(119112, 19)


The pandas [merge_asof](https://pandas.pydata.org/docs/dev/reference/api/pandas.merge_asof.html) method can be used to quickly merge the two sorted datasets into one on the closest time.

In [125]:
m = pd.merge_asof(df, weather_data, on='datetime', direction='nearest')  
m.shape

(4027695, 29)

In [132]:
m.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4027695 entries, 0 to 4027694
Data columns (total 29 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   district               3937463 non-null  category      
 1   nature                 4027695 non-null  category      
 2   status                 4027695 non-null  category      
 3   call_id                4027695 non-null  int64         
 4   datetime               4027695 non-null  datetime64[ns]
 5   isCorner               4027695 non-null  bool          
 6   houseNumber            3426083 non-null  float64       
 7   primaryStreetName      4027695 non-null  category      
 8   primaryStreetSuffix    3844881 non-null  category      
 9   secondaryStreetName    549725 non-null   category      
 10  secondaryStreetSuffix  531853 non-null   category      
 11  loc_id                 4027695 non-null  int64         
 12  isdaytime              40276

In [134]:
m.head()

,district,nature,status,call_id,datetime,isCorner,houseNumber,primaryStreetName,primaryStreetSuffix,secondaryStreetName,...,humidity,visibilityKm,pressureMB,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex
0,NaN,FOLLOW UP,Assignment Completed,163120850,2016-11-07 09:33:52,False,2920.0,N 4TH,ST,NaN,...,64,10,1026,1,14,7,14,18,14,4
1,NaN,PROPERTY DAMAGE,Advised/Referral,163120774,2016-11-07 09:33:55,False,5929.0,W BLUE MOUND,RD,NaN,...,64,10,1026,1,14,7,14,18,14,4
2,NaN,VACANT HOUSE CHK,Assignment Completed,163120851,2016-11-07 09:34:58,False,3425.0,N 93RD,ST,NaN,...,64,10,1026,1,14,7,14,18,14,4
3,NaN,FOLLOW UP,Assignment Completed,163120853,2016-11-07 09:36:20,False,245.0,W LINCOLN,AV,NaN,...,64,10,1026,1,14,7,14,18,14,4
4,NaN,TRAFFIC STOP,Advised,163120857,2016-11-07 09:38:10,False,765.0,N 25TH,ST,NaN,...,64,10,1026,1,14,7,14,18,14,4


In [135]:
m.describe()

,call_id,houseNumber,loc_id,tempC,windspeedKmph,winddirdegree,weatherCode,precipMM,humidity,visibilityKm,pressureMB,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex
count,4.027695e+06,3.426083e+06,4027695.0,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06,4.027695e+06
mean,1.891808e+08,3.367593e+03,1.0,9.332960e+00,1.396831e+01,1.978097e+02,1.490212e+02,8.068526e-02,7.252384e+01,9.280653e+00,1.016711e+03,4.627480e+01,9.741805e+00,4.295812e+00,7.040703e+00,2.030895e+01,7.329460e+00,2.161411e+00
std,1.459309e+07,2.415387e+03,0.0,1.099667e+01,6.506821e+00,9.301290e+01,7.256697e+01,3.459743e-01,1.560838e+01,1.861185e+00,7.728984e+00,3.620323e+01,1.150243e+01,1.050146e+01,1.314242e+01,9.506874e+00,1.356433e+01,1.869588e+00
min,1.630815e+08,1.000000e+00,1.0,-3.100000e+01,0.000000e+00,0.000000e+00,1.130000e+02,0.000000e+00,1.600000e+01,0.000000e+00,9.860000e+02,0.000000e+00,-3.000000e+01,-3.400000e+01,-4.400000e+01,0.000000e+00,-4.400000e+01,1.000000e+00
25%,1.732129e+08,1.614000e+03,1.0,1.000000e+00,9.000000e+00,1.260000e+02,1.130000e+02,0.000000e+00,6.100000e+01,1.000000e+01,1.012000e+03,1.100000e+01,1.000000e+00,-4.000000e+00,-3.000000e+00,1.300000e+01,-3.000000e+00,1.000000e+00
50%,1.906707e+08,2.920000e+03,1.0,9.000000e+00,1.300000e+01,2.140000e+02,1.160000e+02,0.000000e+00,7.400000e+01,1.000000e+01,1.016000e+03,3.900000e+01,9.000000e+00,4.000000e+00,6.000000e+00,1.900000e+01,6.000000e+00,1.000000e+00
75%,2.016115e+08,4.600000e+03,1.0,1.900000e+01,1.800000e+01,2.760000e+02,1.220000e+02,0.000000e+00,8.500000e+01,1.000000e+01,1.021000e+03,8.300000e+01,1.900000e+01,1.400000e+01,1.900000e+01,2.600000e+01,1.900000e+01,3.000000e+00
max,2.201010e+08,6.460500e+05,1.0,3.500000e+01,4.700000e+01,3.600000e+02,3.890000e+02,1.630000e+01,1.000000e+02,1.000000e+01,1.048000e+03,1.000000e+02,4.100000e+01,2.700000e+01,3.500000e+01,7.100000e+01,4.100000e+01,9.000000e+00


// TODO: describe the features in m.
// TODO: update description of weather dataset and where it came from
// TODO: note down research questions and hypotheses at the top of the notebook